# Title: Development of an Automatic Sentiment Analysis Tool for Urdu Text on Social Media Platforms

# Phase 1: Text Preprocessing for Urdu Text

Step 1: Import Libraries and Load Data

In [ ]:
import pandas as pd
from LughaatNLP import LughaatNLP
processor = LughaatNLP()

# Load the dataset
data = pd.read_csv('urdu_sarcastic_dataset.csv')
print(data.head(50))


                                            urdu_text  is_sarcastic  \
0   🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...           1.0   
1   چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...           1.0   
2   کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...           0.0   
3                                        نہیں پائین 😎           0.0   
4    `` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...           1.0   
5         قابل اعتبار ہی اکثر قاتل اعتبار ہوتے ہیں 💔🔥           1.0   
6       انساں کو تھکا دیتا ہے سوچوں کا سفر بھی ... 🍁🥀           0.0   
7                               حامد میر صاحب ویلڈن👏😊           0.0   
8   یار وچارہ ویلا ہوندا ہے اس آرے لگا ہویا ہے😂😂 ت...           1.0   
9            یہ سمجھتے ہیں سارا پاکستان بیوقوف ھے 😂😂😂           1.0   
10                      تسی لڑاںٔی کروانی ساڈی کی 😂😂😂           0.0   
11                       پائن دوبارہ فالو کرئیے..؟؟😆🙄           0.0   
12  کتنی مہنگائی ہے الو دوسو روپے درجن کدو 80روپے ...           1.0   
13   😍

/Applications/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Step 2: Define Custom Stopwords

Step 3: Create Stopword Removal Function

In [ ]:
sentiment_words = [
    'نہیں', 'برا', 'اچھا', 'خوش', 'غمگین', 'محبت', 'نفرت',
    'خوشی', 'مایوسی', 'شکریہ', 'تکلیف', 'خوف', 'کامیابی',
    'ناکامی', 'امید', 'خواب', 'صبر', 'غصہ', 'شرم', 'عزت',
    'تعجب', 'تشکر', 'دوستی', 'تجربہ', 'رنج', 'سکون',
    'محبت', 'غمی', 'مسکراہٹ', 'خوشبو', 'غفلت', 'آزادی',
    'دوستی'
]
sentiment_set = set(sentiment_words)

def remove_stopwords(text, sentiment_stopwords):
   filtered_text = processor.remove_stopwords(text)
    # Retain sentiment-carrying stopwords
   filtered_text = " ".join([word for word in filtered_text.split() if word not in sentiment_stopwords])

   return filtered_text

data['cleaned_text'] = data['urdu_text'].apply(
    lambda x: remove_stopwords(x, sentiment_set) if pd.notnull(x) else x
)

print(data[['urdu_text', 'cleaned_text']].head())



                                           urdu_text  \
0  🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...   
1  چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...   
2  کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...   
3                                       نہیں پائین 😎   
4   `` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...   

                                        cleaned_text  
0                 🤣😂😂 لینے شادی فسادن ٹھیک کوجی 😐😐😐🤣  
1   چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں😂😂  
2  کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...  
3                                            پائین 😎  
4       `` مراد علی شاہ بھیس ڈی آئی آئی '' حامد میر😁  


Step 4: Remove unnecessary punctuation, URLs, hashtags, and handle emojis appropriately.

In [ ]:
import re
import string
import emoji

# Emoji sentiment dictionary
emoji_sentiment_dict = {
    '😊': 'positive',
    '😞': 'negative',
    '❤️': 'positive',
    '📚': 'neutral',
    '😭': 'negative',
    '🌹': 'positive',
    '😂': 'positive',
    '😢': 'negative',
    '🥺': 'negative',
    '😡': 'negative',
    '😍': 'positive',
    '👍': 'positive',
    '👎': 'negative',
    '🎉': 'positive',
    '😱': 'negative',
    '😌': 'positive',
    '😏': 'neutral',
    '🤔': 'neutral',
    '🙌': 'positive',
    '🤗': 'positive',
    '💔': 'negative',
    '🌈': 'positive',
    '☹️': 'negative',
    '🥳': 'positive',
    '😎': 'positive',
    '😴': 'neutral',
    '😋': 'positive',
    '🤑': 'positive',
    '🤩': 'positive',
    '🤝': 'positive',
    '😬': 'neutral',
    '🙈': 'neutral',
    '🌍': 'neutral',
    '🦋': 'positive',
    '🥲': 'positive',
    '😜': 'positive',
    '🙄': 'negative',
    '😆': 'positive',
    '👏': 'positive',
    '🤣': 'positive',
    '✌️': 'neutral',
    '😕': 'neutral'


}

def clean_text(text):
    if not isinstance(text, str):  # Check if the input is a string
        return ''  # Return an empty string if it's not

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)

    text = re.sub(r"#\w+", '', text)
    # Remove punctuations (except those in the sentiment dict)
    text = text.translate(str.maketrans('', '', string.punctuation))

    cleaned_text = []
    for char in text:
        if char in emoji_sentiment_dict:
            cleaned_text.append(emoji_sentiment_dict[char])
        elif not emoji.is_emoji(char):
            cleaned_text.append(char)  #

    return ''.join(cleaned_text)

# Apply cleaning function
data['cleaned_text1'] = data['cleaned_text'].apply(clean_text)

# Print the results
print(data['cleaned_text1'].head(50))

0     positivepositivepositive لینے شادی فسادن ٹھیک ...
1     چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...
2     کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
3                                        پائین positive
4                مراد علی شاہ بھیس ڈی آئی آئی  حامد میر
5                      قابل اعتبار قاتل اعتبار negative
6                              انساں تھکا سوچوں سفر    
7                        حامد میر ویلڈنpositivepositive
8     یار وچارہ ویلا ہوندا آرے ہویا ہےpositivepositi...
9     سمجھتے سارا پاکستان بیوقوف ھے positivepositive...
10      تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11        پائن دوبارہ فالو کرئیے   ؟ ؟ positivenegative
12    کتنی مہنگائی الو دوسو روپے درجن کدو 80روپے گز ...
13    positiveعشق راس آۓ negativeزخم کھاٶ گے positiv...
14                                        چونا positive
15    خاتم  النبیین  محمدﷺ Surat 73 سورة المزمل Ayt ...
16                بیچارے بیویاں ہیے تیسری ٹرائ positive
17                پتہ ہورہا بورڈ زنگ ہےnegativen

Step 5: Handle Short Conversations : filter out very short posts or those with less than three words

In [ ]:

def filter_short_posts(text):
    return len(text.split()) >= 3


data['filtered_text'] = data['cleaned_text1'].apply(lambda x: x if filter_short_posts(x) else None)

# Drop rows with None values
data.dropna(subset=['filtered_text'], inplace=True)
print(data['filtered_text'].head(50))


0     positivepositivepositive لینے شادی فسادن ٹھیک ...
1     چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...
2     کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
4                مراد علی شاہ بھیس ڈی آئی آئی  حامد میر
5                      قابل اعتبار قاتل اعتبار negative
6                              انساں تھکا سوچوں سفر    
7                        حامد میر ویلڈنpositivepositive
8     یار وچارہ ویلا ہوندا آرے ہویا ہےpositivepositi...
9     سمجھتے سارا پاکستان بیوقوف ھے positivepositive...
10      تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11        پائن دوبارہ فالو کرئیے   ؟ ؟ positivenegative
12    کتنی مہنگائی الو دوسو روپے درجن کدو 80روپے گز ...
13    positiveعشق راس آۓ negativeزخم کھاٶ گے positiv...
15    خاتم  النبیین  محمدﷺ Surat 73 سورة المزمل Ayt ...
16                بیچارے بیویاں ہیے تیسری ٹرائ positive
17                پتہ ہورہا بورڈ زنگ ہےnegativenegative
18                اللہ رحمتوں سائے مکمل صحت یاب فرمائے 
19     آرمی چیف نوٹس ۔ مجرم جرائیم نوٹس کرےگا ۔ 

In [ ]:
data['urdu_text'] = data['filtered_text']
print("Final Preprocessed Urdu Text:")
print(data[['urdu_text']].head(30))

Final Preprocessed Urdu Text:
                                            urdu_text
0   positivepositivepositive لینے شادی فسادن ٹھیک ...
1   چل مہمانوں کھانا سرو چڑیل چاچی نوں دسدی آں میں...
2   کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
4              مراد علی شاہ بھیس ڈی آئی آئی  حامد میر
5                    قابل اعتبار قاتل اعتبار negative
6                            انساں تھکا سوچوں سفر    
7                      حامد میر ویلڈنpositivepositive
8   یار وچارہ ویلا ہوندا آرے ہویا ہےpositivepositi...
9   سمجھتے سارا پاکستان بیوقوف ھے positivepositive...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11      پائن دوبارہ فالو کرئیے   ؟ ؟ positivenegative
12  کتنی مہنگائی الو دوسو روپے درجن کدو 80روپے گز ...
13  positiveعشق راس آۓ negativeزخم کھاٶ گے positiv...
15  خاتم  النبیین  محمدﷺ Surat 73 سورة المزمل Ayt ...
16              بیچارے بیویاں ہیے تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18              اللہ رحمتوں سائے مکمل صحت یاب فرمائے

# Phase 2: Stemming and Lemmatization for Urdu Text

STEMMING

In [ ]:
def urdu_stemmer(text):
    if isinstance(text, str):  # Check if the input is a string
        stemmed_text = processor.urdu_stemmer(text)
        return stemmed_text
    return text

In [ ]:
data['stemmed_text'] = data['urdu_text'].apply(urdu_stemmer)

# Step 6: Print the results
print(data[[ 'stemmed_text']].head(50))

                                         stemmed_text
0   positivepositivepositive لینہ شادی فسادن ٹھیک ...
1   چل مہمانا کھانا سرو چڑیل چاچی نا دسدی آں میںpo...
2   کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
4               مراد علی شاہ بھیس ڈی آئی آئی حامد میر
5                      قابل اعتبر قاتل اعتبر negative
6                                 انساں تھکا سوچا سفر
7                      حامد میر ویلڈنpositivepositive
8   یار وچارہ ویلا ہوندا آرہ ہویا ہےpositivepositi...
9   سمجھتہ سارا پاکستن بیوقوف ھہ positivepositivep...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11         پائن دوبرہ فالو کرئیہ ؟ ؟ positivenegative
12  کتنی مہنگائی الو دوسو روپہ درجن کدو 80روپہ گز ...
13  positiveعشق راس آۓ negativeزخم کھاٶ گہ positiv...
15  خاتم النبیین محمدﷺ Surat 73 سوره المزمل Ayt 20...
16                بیچارہ بیوی ہیہ تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18                 اللہ رحمت سائہ مکمل صحت یاب فرمائہ
19   آرمی چیف نوٹس ۔ مجرم جر

In [ ]:
data['urdu_text'] = data['stemmed_text']
print("Final Preprocessed Urdu Text:")
print(data[['urdu_text']].head(30))

Final Preprocessed Urdu Text:
                                            urdu_text
0   positivepositivepositive لینہ شادی فسادن ٹھیک ...
1   چل مہمانا کھانا سرو چڑیل چاچی نا دسدی آں میںpo...
2   کامران خان دن بھریہ زمہ داری لگائی گئی اپوزیشن...
4               مراد علی شاہ بھیس ڈی آئی آئی حامد میر
5                      قابل اعتبر قاتل اعتبر negative
6                                 انساں تھکا سوچا سفر
7                      حامد میر ویلڈنpositivepositive
8   یار وچارہ ویلا ہوندا آرہ ہویا ہےpositivepositi...
9   سمجھتہ سارا پاکستن بیوقوف ھہ positivepositivep...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11         پائن دوبرہ فالو کرئیہ ؟ ؟ positivenegative
12  کتنی مہنگائی الو دوسو روپہ درجن کدو 80روپہ گز ...
13  positiveعشق راس آۓ negativeزخم کھاٶ گہ positiv...
15  خاتم النبیین محمدﷺ Surat 73 سوره المزمل Ayt 20...
16                بیچارہ بیوی ہیہ تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18                 اللہ رحمت سائہ مکمل صحت یاب فرمائ

LEMMATIZATION

In [ ]:
def urdu_lemmatizer(text):
    if isinstance(text, str):
        lemmatized_text = processor.lemmatize_sentence(text)
        return lemmatized_text
    return text

In [ ]:
data['lemmatized_text'] = data['urdu_text'].apply(urdu_lemmatizer)


print(data[['lemmatized_text']].head(50))

                                      lemmatized_text
0   positivepositivepositive لینہ شادی فسادن ٹھیک ...
1   چلنا مہمانا کھا سرا چڑیل چاچی نا دسدی آں میںpo...
2   کامران خان دن بھریہ زمہ داری لگنا جانا اپوزیشن...
4               مراد علی شاہ بھیس ڈی آئی آئی حامد میر
5                      قابل اعتبر قاتل اعتبر negative
6                               انساں تھکنا سوچنا سفر
7                      حامد میر ویلڈنpositivepositive
8   یار وچارہ ویلا ہوندا آرہ ہویا ہےpositivepositi...
9   سمجھتہ سارا پاکستن بیوقوف ھہ positivepositivep...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11         پائن دوبرہ فالو کرئیہ ؟ ؟ positivenegative
12  کتنا مہنگائی الہ دوسو روپہ درجن کدہ 80روپہ گز ...
13  positiveعشق راس آۓ negativeزخم کھاٶ گہ positiv...
15  خاتم النبیین محمدﷺ Surat 73 سوره المزمل Ayt 20...
16                بیچارہ بیوی ہیہ تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18                 اللہ رحمت سائہ مکمل صحت یاب فرمائہ
19   آرمی چیف نوٹس ۔ مجرم جر

In [ ]:
data['urdu_text'] = data['lemmatized_text']
print("Final Preprocessed Urdu Text:")
print(data[['urdu_text']].head(30))

Final Preprocessed Urdu Text:
                                            urdu_text
0   positivepositivepositive لینہ شادی فسادن ٹھیک ...
1   چلنا مہمانا کھا سرا چڑیل چاچی نا دسدی آں میںpo...
2   کامران خان دن بھریہ زمہ داری لگنا جانا اپوزیشن...
4               مراد علی شاہ بھیس ڈی آئی آئی حامد میر
5                      قابل اعتبر قاتل اعتبر negative
6                               انساں تھکنا سوچنا سفر
7                      حامد میر ویلڈنpositivepositive
8   یار وچارہ ویلا ہوندا آرہ ہویا ہےpositivepositi...
9   سمجھتہ سارا پاکستن بیوقوف ھہ positivepositivep...
10    تسی لڑاںٔی کروانی ساڈی positivepositivepositive
11         پائن دوبرہ فالو کرئیہ ؟ ؟ positivenegative
12  کتنا مہنگائی الہ دوسو روپہ درجن کدہ 80روپہ گز ...
13  positiveعشق راس آۓ negativeزخم کھاٶ گہ positiv...
15  خاتم النبیین محمدﷺ Surat 73 سوره المزمل Ayt 20...
16                بیچارہ بیوی ہیہ تیسری ٹرائ positive
17              پتہ ہورہا بورڈ زنگ ہےnegativenegative
18                 اللہ رحمت سائہ مکمل صحت یاب فرمائ

# Phase 3: Feature Extraction from Urdu Text

 Tokenization

In [ ]:
def urdu_tokenize(text):
    if isinstance(text, str):  # Check if the input is a string
        tokens = processor.urdu_tokenize(text)  # Tokenize using the LughatNLP processor
        return tokens
    return text

In [ ]:
data['tokens'] = data['urdu_text'].apply(urdu_tokenize)
print(data[[ 'tokens']].head(50))

                                               tokens
0                     [لینہ, شادی, فسادن, ٹھیک, کوجی]
1   [چلنا, مہمانا, کھا, سرا, چڑیل, چاچی, نا, دسدی,...
2   [کامران, خان, دن, بھریہ, زمہ, داری, لگنا, جانا...
4     [مراد, علی, شاہ, بھیس, ڈی, آئی, آئی, حامد, میر]
5                          [قابل, اعتبر, قاتل, اعتبر]
6                          [انساں, تھکنا, سوچنا, سفر]
7                                  [حامد, میر, ویلڈن]
8   [یار, وچارہ, ویلا, ہوندا, آرہ, ہویا, ہے, تسی, ...
9                  [سمجھتہ, سارا, پاکستن, بیوقوف, ھہ]
10                        [تسی, لڑاںٔی, کروانی, ساڈی]
11                   [پائن, دوبرہ, فالو, کرئیہ, ؟, ؟]
12  [کتنا, مہنگائی, الہ, دوسو, روپہ, درجن, کدہ, 80...
13         [عشق, راس, آۓ, زخم, کھاٶ, گہ, مُسکراٶ, گہ]
15  [خاتم, النبیین, محمدﷺ, 73, سوره, المزمل, 20, ح...
16                   [بیچارہ, بیوی, ہیہ, تیسری, ٹرائ]
17                        [پتہ, ہورہا, بورڈ, زنگ, ہے]
18         [اللہ, رحمت, سائہ, مکمل, صحت, یاب, فرمائہ]
19  [آرمی, چیف, نوٹس, ۔, مجر

 Import Required Libraries

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

Tf-IDF (Term Frequency-Inverse Document Frequency):

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


#
corpus = data['urdu_text'].dropna().tolist()  # Drop any NaN values

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Get the feature names (words) and their corresponding TF-IDF scores
feature_names = tfidf_vectorizer.get_feature_names_out()
dense = tfidf_matrix.todense()

# Create a DataFrame with TF-IDF scores
tfidf_df = pd.DataFrame(dense, columns=feature_names)

mean_tfidf_scores = tfidf_df.mean(axis=0).sort_values(ascending=False)

# Get the top 10 words with the highest TF-IDF scores
top_tfidf_words = mean_tfidf_scores.head(10).reset_index()
top_tfidf_words.columns = ['Word', 'Mean_TF_IDF_Score']

# Display the results
print("\nTop 10 Words with Highest TF-IDF Scores:")
print(top_tfidf_words)



Top 10 Words with Highest TF-IDF Scores:
                       Word  Mean_TF_IDF_Score
0                  positive           0.027201
1          positivepositive           0.013750
2  positivepositivepositive           0.011043
3                      کوئی           0.010329
4                      اللہ           0.010233
5                       خان           0.010041
6                        ھہ           0.009844
7                        به           0.009398
8                      سندھ           0.007996
9                        نا           0.007666


 Word2Vec:

In [ ]:
# Prepare the tokenized text for Word2Vec training
tokenized_texts =data['tokens'].tolist()

# Train the Word2Vec model using the tokenized text
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=8, min_count=2, workers=4)

# List the top 5 words most similar to the word "اچھا" (good)
try:
    similar_words = word2vec_model.wv.most_similar("اچھا", topn=5)
    print("\nTop 5 Words Most Similar to 'اچھا':")
    for word, similarity in similar_words:
        print(f"{word}: {similarity:.4f}")
except KeyError:
    print("\nThe word 'اچھا' is not in the vocabulary.")


Top 5 Words Most Similar to 'اچھا':
دیکھتہ: 0.9307
واکا: 0.9306
ہورہی: 0.9299
ڈیزل: 0.9296
کاکول: 0.9289


In [ ]:

# Assuming 'tokens' column contains tokenized text
tokenized_texts = data['tokens'].tolist()

# Print sample of tokenized texts to inspect
print("Sample Tokenized Texts:")
for tokens in tokenized_texts[:5]:
    print(tokens)

# Train the Word2Vec model using the tokenized text with min_count=1
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Check vocabulary size and sample
print("Vocabulary Size:", len(word2vec_model.wv.key_to_index))
print("Vocabulary Sample:", list(word2vec_model.wv.key_to_index)[:20])

# Find the top 5 words most similar to 'اچھا' (good)
try:
    similar_words = word2vec_model.wv.most_similar("اچھا", topn=5)
    print("\nTop 5 Words Most Similar to 'اچھا':")
    for word, similarity in similar_words:
        print(f"{word}: {similarity:.4f}")
except KeyError:
    print("\nThe word 'اچھا' is not in the vocabulary.")


Sample Tokenized Texts:
['لینہ', 'شادی', 'فسادن', 'ٹھیک', 'کوجی']
['چلنا', 'مہمانا', 'کھا', 'سرا', 'چڑیل', 'چاچی', 'نا', 'دسدی', 'آں', 'میں']
['کامران', 'خان', 'دن', 'بھریہ', 'زمہ', 'داری', 'لگنا', 'جانا', 'اپوزیشن', 'کردار', 'کشی', 'اوراس', 'پربھونکناہےآپ', 'خوشامدگری', 'وچاپلوسی', 'سےاورکتنی', 'دولت', 'کماناچاہتےہا', 'موٹرسائیکل', 'سےپیجارو', 'پراڈو', 'کےسفرما', 'ضمیرکی', 'لاش', 'سےاٹھتی', 'بدبوآپ', 'ناک', 'نوٹ', 'سےالتجاگزارش', 'ہےہما', 'فالوکرا']
['مراد', 'علی', 'شاہ', 'بھیس', 'ڈی', 'آئی', 'آئی', 'حامد', 'میر']
['قابل', 'اعتبر', 'قاتل', 'اعتبر']
Vocabulary Size: 20801
Vocabulary Sample: ['۔', '؟', '️', 'کوئی', 'خان', 'ھہ', 'اللہ', 'به', 'پاکستن', 'سندھ', 'گہ', 'کہنا', 'آئی', 'نواز', 'عمران', 'حکومت', 'لوگ', 'اہ', 'تنقید', '…']

Top 5 Words Most Similar to 'اچھا':
ٹرانسپورٹ: 0.8969
وہم: 0.8958
اون: 0.8956
ذیادہ: 0.8955
ﺭﮨﻨﺎ: 0.8955


# Phase 4: N-grams Analysis

In [ ]:
from nltk.util import ngrams
from collections import Counter

UNIGRAMS:

In [ ]:
tokenized_texts = data['tokens'].tolist()  # Tokenized Urdu text as lists of words

# Flatten the tokenized texts for unigram analysis
all_words = [word for tokens in tokenized_texts for word in tokens]

# Unigrams
unigram_freq = Counter(all_words)


In [ ]:
top_unigrams = unigram_freq.most_common(10)
print("\nTop 10 Unigrams:")
for word, freq in top_unigrams:
    print(f"{word}: {freq}")



Top 10 Unigrams:
۔: 11567
؟: 1582
️: 1555
خان: 1172
کوئی: 1172
ھہ: 1066
اللہ: 996
به: 976
پاکستن: 859
سندھ: 815


BIGRAMS:

In [ ]:
bigram_list = [list(ngrams(tokens, 2)) for tokens in tokenized_texts]
bigrams = [bigram for sublist in bigram_list for bigram in sublist]  # Flatten
bigram_freq = Counter(bigrams)


In [ ]:
top_bigrams = bigram_freq.most_common(10)
print("\nTop 10 Bigrams:")
for bigram, freq in top_bigrams:
    print(f"{bigram}: {freq}")



Top 10 Bigrams:
('۔', '۔'): 4706
('عمران', 'خان'): 508
('؟', '؟'): 496
('نواز', 'شریف'): 448
('\u200d', '️'): 332
('سندھ', 'پولیس'): 307
('️', '️'): 260
('آرمی', 'چیف'): 223
('کیپٹن', 'صفدر'): 179
('اردو', 'زبن'): 172


TRIGRAMS:

In [ ]:
trigram_list = [list(ngrams(tokens, 3)) for tokens in tokenized_texts]
trigrams = [trigram for sublist in trigram_list for trigram in sublist]  # Flatten
trigram_freq = Counter(trigrams)


In [ ]:
top_trigrams = trigram_freq.most_common(10)
print("\nTop 10 Trigrams:")
for trigram, freq in top_trigrams:
    print(f"{trigram}: {freq}")


Top 10 Trigrams:
('۔', '۔', '۔'): 2310
('؟', '؟', '؟'): 234
('\u2066', '️', '\u2069'): 130
('️', '️', '️'): 121
('\u200d', '️', '\u200d'): 117
('️', '\u200d', '️'): 116
('پینا', 'ٹی', 'آئی'): 114
('پینا', 'ڈی', 'ایم'): 88
('صلی', 'اللہ', 'علیہ'): 88
('جزاک', 'اللہ', 'خیر'): 74


# Phase 5: Sentiment Classification Model

using tf-idf matrix calculated above with NAIVE BAYES CLASSIFIER

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'tfidf_matrix' contains your features and 'data' has the labels
X = tfidf_matrix
y = data['is_sarcastic']  # replace with your actual label column

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = nb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.7571
Precision: 0.7067
Recall: 0.9092
F1 Score: 0.7953


# Phase 6

# 1. Evaluation

# Model Performance

The sentiment analysis model achieved an accuracy of 75.71%, with precision at 70.67%, recall at 90.92%, and an F1 score of 79.53%. These metrics suggest that while the model is effective at identifying positive sentiments, it tends to miss some negative sentiments, leading to a lower precision score.

# Challenges with Stemming and Lemmatization



One significant challenge is stemming and lemmatization. The rich morphology of Urdu makes it difficult to identify root forms of words accurately. This can result in the model misinterpreting meanings, especially in complex sentences or sarcastic expressions, which require a deeper understanding of context.

# Areas for Improvement


To enhance the model's performance, we should refine preprocessing techniques for better handling of word variations. Additionally, integrating contextual embeddings or pre-trained models that understand the nuances of Urdu could lead to improvements in identifying sentiments accurately.


# 2. Challenges in Urdu Sentiment Analysis

# Complex Morphology
Urdu’s intricate word structure complicates stemming and lemmatization processes. This complexity can lead to the loss of essential context during text analysis, resulting in inaccurate sentiment classification.


# Colloquial Language
The informal nature of language used on social media introduces slang, abbreviations, and unique expressions, which are often absent in formal Urdu. This variation makes it challenging for the model to capture sentiments accurately.

# Noisy Data
Social media platforms often contain noisy data, including emojis, misspellings, and unconventional grammar. Such noise can hinder the model’s ability to extract clear sentiment signals.


# Optimizing the NLP Pipeline
To improve our Urdu sentiment analysis, we should focus on better data normalization techniques and expand the training dataset to include a wider range of colloquial expressions. Utilizing advanced language models that better capture context will also enhance the model's accuracy and robustness in sentiment classification.